# Test Classification vs Regression and different model frameworks

In [2]:
import sys, os, shutil
import numpy as np
import pandas as pd
import tensorflow as tf

# Add src to path
if not '../src' in sys.path:
    sys.path.insert(0, '../src')

In [3]:
## Add src modules
from Cross_Validator import Cross_Validator as CV
from Strategy import Strategy
from NumTForm import Transformer

In [3]:
strats = []
for b in np.linspace(1, 1.1, 10):
    for s in np.linspace(0.9, 1, 10):
        for m in np.linspace(0.1, 0.5, 10):
            strats.append(Strategy(100000, b, s, m))

### Cross Validate A Regression LSTM

In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(100, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-5)))
model.add(tf.keras.layers.LSTM(50, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-5)))
model.add(tf.keras.layers.LSTM(30, kernel_regularizer=tf.keras.regularizers.l2(1e-5)))
model.add(tf.keras.layers.Dense(30, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
model.add(tf.keras.layers.Dense(30, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
model.add(tf.keras.layers.Dense(30, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError())


In [ ]:
cv = CV(
    start_year=1990,
    end_year = 2000,
    test_size=0.2,
    model=model,
    strats=None,
    epochs=30,
    iters=8
)
cv.run()

### Cross Validate A Classification LSTM

In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(100, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
model.add(tf.keras.layers.LSTM(30, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
model.add(tf.keras.layers.Dense(30, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

In [ ]:
cv_bin = CV(
    start_year=1990,
    end_year = 2000,
    test_size=0.2,
    model=model,
    strats=None,
    epochs=30,
    batch_size=128,
    iters=5,
    binary=True,
    out_dir=os.path.join('..', 'data_files', 'cv_bin_results2')
)
cv_bin.run()

## Cross Validate A Transformer

In [5]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


num_layers = 6
d_model = 128
num_heads = 8
dff = 512

tform = Transformer(
            num_layers = num_layers,
            d_model = d_model,
            num_heads = num_heads,
            dff = dff)

learning_rate = CustomSchedule(d_model)
tform.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, 
                                            beta_2=0.98, epsilon=1e-9),
        loss=tf.keras.losses.MeanSquaredError())

In [7]:
cv_trans = CV(
    start_year=1990,
    end_year = 2000,
    test_size=0.2,
    model=tform,
    strats=None,
    epochs=30,
    batch_size=128,
    iters=5,
    binary=True,
    out_dir=os.path.join('..', 'data_files', 'cv_bin_results_trans')
)
cv_trans.run()

Epoch 1/30


InvalidArgumentError:  required broadcastable shapes
	 [[node mean_squared_error_1/SquaredDifference
 (defined at /opt/anaconda3/lib/python3.8/site-packages/keras/losses.py:1204)
]] [Op:__inference_train_function_29406]

Errors may have originated from an input operation.
Input Source operations connected to node mean_squared_error_1/SquaredDifference:
In[0] transformer/encoder/encoder_layer/multi_head_attention/Softmax (defined at ../src/NumTForm.py:30)	
In[1] IteratorGetNext (defined at /opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:866)

Operation defined at: (most recent call last)
>>>   File "/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
